 # Simulating CH4 with PennyLane 

## Building molecular Hamiltonian 

In [ ]:
from pennylane import numpy as np 
from pennylane import qchem 
import pennylane as qml 
import time 

# defining the molecule 

symbols = ["C", "H", "H", "H", "H"]
coordinates = 0.529*np.array([[0.0,0.0,0.0],[0.6276,0.6276,0.6276],[0.6276,-0.6276,-0.6276],[-0.6276,0.6276,-0.6276],[-0.6276,-0.6276,0.6276]])
# coordinates in atomic units 

In [ ]:
# defining molecule specificites for simplification after having studied it's MOs

charge = 0  # not an ion 
mult = 1 # initially, prone to change if Givens rotations 
active_electrons = 8 # not considering the 1s core electrons 
active_orbitals = 5 # neglecting core 1s and the 3 MOs with the highest energy

H, qubits = qchem.molecular_hamiltonian(symbols,coordinates,charge=charge, mult = mult, active_electrons = active_electrons, active_orbitals = active_orbitals)


## Energy and ground state 

In [ ]:
electrons = 8
hf = qml.qchem.hf_state(electrons, qubits) # creating corresponding Hartree-Fock state
print(hf)

### Evaluating the relevant Givens rotations (i.e. electrons excitations)

In [ ]:
singles, doubles = qchem.excitations(active_electrons,qubits)
print("Total number of excitations = {}".format(len(singles)+len(doubles))) # yields 24 possible spin-preserving excitations

def circuit_1(params, excitations): 
    qml.BasisState(hf,wires=range(qubits)) # generating the corresponding Hartree-Fock state in the the VQC
    for i, excitation in enumerate(excitations):
        if len(excitation) == 4: # meaning a double-excitation 
            qml.DoubleExcitation(params[i],wires=excitation) # where the excitation is being applied
        else: # meaning single-excitation 
            qml.SingleExcitation(params[i],wires=excitation) 
    return qml.expval(H)

""" 
Protocol : 

- Compute gradients for all double excitations.

- Select the double excitations with gradients larger than a pre-defined threshold.

- Perform VQE to obtain the optimized parameters for the selected double excitations.

- Repeat steps 1 and 2 for the single excitations.

- Perform the final VQE optimization with all the selected excitations.
"""

In [ ]:
# double excitations selection

dev = qml.device("default.qubit",wires=qubits)
cost_fn = qml.QNode(circuit_1,dev) # instead of the decorator, same thing otherwise

circuit_gradient = qml.grad(cost_fn,argnum=0) # returns the gradient as a callable function of (functions of) QNodes.

params = [0.0] * len(doubles) # parameter values to zero such that the gradients are computed with respect to the Hartree-Fock state.
grads = circuit_gradient(params,excitations=doubles) # OK 

#for i in range(len(doubles)):
    #print(f"Excitation : {doubles[i]}, Gradient : {grads[i]}")

In [ ]:
# defining the right threshold in order not to have to many Givens rotations 

doubles_select = [doubles[i] for i in range(len(doubles)) if abs(grads[i]) > 1.0e-2]
# len(doubles_select), if 1.0e-3 takes all the excitations

In [ ]:
# finding the optimizing parameters for double excitations 

opt = qml.GradientDescentOptimizer(stepsize=0.5) # QNG doesn't seem to provide any speed-up
params_doubles = np.zeros(len(doubles_select),requires_grad = True) 

for n in range(20): # number of optimizations 
    params_doubles = opt.step(cost_fn, params_doubles, excitations=doubles_select) 

In [ ]:
# single excitations selection (same idea but have to prior consider the previously selected double excitations)

def circuit_2(params, excitations, gates_select, params_select):
    qml.BasisState(hf, wires=range(qubits))
    for i, gate in enumerate(gates_select): # applying the selected double excitations 
        qml.DoubleExcitation(params_select[i], wires=gate)       
    for i, gate in enumerate(excitations): # testing the single excitations 
        qml.SingleExcitation(params[i], wires=gate)
    return qml.expval(H)

cost_fn = qml.QNode(circuit_2, dev)
circuit_gradient = qml.grad(cost_fn, argnum=0)
params = [0.0] * len(singles)

grads = circuit_gradient(
    params,
    excitations=singles,
    gates_select=doubles_select,
    params_select=params_doubles
)

#for i in range(len(singles)):
    #print(f"Excitation : {singles[i]}, Gradient: {grads[i]}") # f format string

In [ ]:
# defining the right threshold in order not to have to many Givens rotations 

singles_select = [singles[i] for i in range(len(singles)) if abs(grads[i]) > 1.0e-4]
# len(singles_select)

In [ ]:
# applying all selected excitations and VQE to optimize the full quantum circuit 

# before, to speed things up, let's consider the sparsing (lot of zeroes) of the molecular Hamiltonian 

H_sparse = qml.utils.sparse_hamiltonian(H)

# now the VQE 

opt = qml.GradientDescentOptimizer(stepsize=0.5) # what about QNG ?

excitations = doubles_select + singles_select

params = np.zeros(len(excitations), requires_grad=True)

@qml.qnode(dev, diff_method="parameter-shift")
def circuit(params):
    qml.BasisState(hf, wires=range(qubits))

    for i, excitation in enumerate(excitations):
        if len(excitation) == 4:
            qml.DoubleExcitation(params[i], wires=excitation)
        elif len(excitation) == 2:
            qml.SingleExcitation(params[i], wires=excitation)

    return qml.expval(qml.SparseHamiltonian(H_sparse, wires=range(qubits)))


for n in range(20):
    t1 = time.time()
    params, energy = opt.step_and_cost(circuit, params)
    t2 = time.time()
    print("n = {:},  E = {:.8f} H, t = {:.2f} s".format(n, energy, t2 - t1))

In [ ]:
# comparing to automatically found eigenvalues :
Hs = qml.SparseHamiltonian(qml.utils.sparse_hamiltonian(H), wires=range(qubits))
qml.eigvals(Hs,k=5) # same order of magnitude !